In [1]:
import imageio
import torch
import numpy as np
from ripser import ripser, lower_star_img
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import cv2
from matplotlib import pyplot as plt
import numpy as np # linear algebra
import pandas as pd # reading and processing of tables
import skimage, os
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
# import dicom
import scipy.misc
from tqdm import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
!ls

COVID           article.ipynb   labels.pkl      non-COVID       transformed.npy


In [3]:
covid_images = !ls COVID/
non_covid_images = !ls non-COVID/

In [4]:
covid_images = ["COVID/" + x for x in covid_images]
non_covid_images = ["non-COVID/" + x for x in non_covid_images]
len(covid_images), len(non_covid_images)

(1252, 1229)

In [5]:
img = cv2.imread(covid_images[0], cv2.IMREAD_GRAYSCALE)
lower_star_img(img)

array([[ 37.,  38.],
       [ 37.,  38.],
       [ 38.,  39.],
       ...,
       [239., 255.],
       [199., 255.],
       [ 36.,  inf]])

In [22]:
%%time
non_covid_diagrams = [lower_star_img(cv2.imread(x, cv2.IMREAD_GRAYSCALE))
                      for x in tqdm(non_covid_images)]
covid_diagrams = [lower_star_img(cv2.imread(x, cv2.IMREAD_GRAYSCALE))
                      for x in tqdm(covid_images)]

100%|██████████| 1252/1252 [03:32<00:00,  5.90it/s]

CPU times: user 6min 39s, sys: 50.6 s, total: 7min 29s
Wall time: 7min 31s


In [34]:
diagrams = non_covid_diagrams + covid_diagrams
labels = [0] * len(non_covid_diagrams) + [1] * len(covid_diagrams)

In [35]:
import random
c = list(zip(diagrams, labels))

random.shuffle(c)

diagrams, labels = zip(*c)

In [36]:
diagrams = list(diagrams)

def clean_dgms(dgms):
    for i in tqdm(range(len(dgms))):
        array = dgms[i]
        whereinf = np.isfinite(array)
        array[~whereinf] = 1
        array = np.nan_to_num(array)
        dgms[i] = array
    return dgms
cleaned_dgms = clean_dgms(diagrams)

100%|██████████| 2481/2481 [00:00<00:00, 6303.95it/s]


In [37]:
import gudhi.representations as tda

In [38]:
# with open('diagrams.pickle', 'wb') as f:
#     pickle.dump(diagrams, f)

In [39]:
with open('diagrams.pickle', 'rb') as f:
    diagrams = pickle.load(f)

In [40]:
import pickle 
# with open('labels.pkl', 'wb') as f:
#     pickle.dump(list(labels), f)
with open('labels.pkl', 'rb') as f:
    labels = pickle.load(f)

In [49]:
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

In [63]:
%%time
betti = tda.BettiCurve(resolution=100)
transformed = betti.fit_transform(diagrams)
clf = CatBoostClassifier()
X_train, X_test, y_train, y_test = train_test_split(transformed, labels)

clf.fit(X_train, y_train, verbose=False)
clf.score(X_test, y_test)
clf.save_model('betti+catboost.model')

CPU times: user 50.5 s, sys: 590 ms, total: 51.1 s
Wall time: 20.4 s


0.9855072463768116